**Implementaciones pendientes:**

- Revisar los Bo5 que no pasan bien y como tener el sistema de bans para que tenga sentido en la migracion.

- Revisar nomenclatura de archivos csv, hay cosas hardcodeadas y pueden llevar a un error.

- Revisar Link extractor y porque no funciona en ciertos eventos esto funciona, falta testeo.

- Error si no tiene los picks y bans

- Siento que se hacen 2 veces algunas cosas, en especial desde la implementacion de detalle de rondas. Ej: Calculo de ronda de pistols o la suma de las rondas por lado.

- Stats por player

**Funcionalidades:**

Con un link de matches de VLR, extraer la seguiente info de la partida

- Picks, bans y si es equipo A o B
- Rondas ganadas por lado
- Pistols ganadas por lado
- Resultado ronda por ronda y tipo de win
- Fecha de la partida


In [3]:
from bs4 import BeautifulSoup
from urllib.request import urlopen
import csv
import re
import time
import random
import pandas as pd
from itertools import repeat

In [44]:
# corrector de longitud de fila
def lenCorrector(rowInfo):
    evalList = [
        "own_pistol_tt",
        "own_pistol_ct",
        "own_score_tt",
        "rival_score_tt",
        "own_score_ct",
        "rival_score_ct",
    ]
    for col in evalList:
        if len(rowInfo["map"]) > len(rowInfo[col]):
            rowInfo[col].append(0)
    if len(rowInfo["map"]) > len(rowInfo["team"]):
        rowInfo["team"].append(rowInfo["team"][len(rowInfo["team"]) - 1])
        rowInfo["rival"].append(rowInfo["rival"][len(rowInfo["rival"]) - 1])

In [45]:
# Individual info extractors


def event_name(soup):

    event = soup.find("title").get_text(strip=True)
    resultado = re.search(r"\|([^|]+)\|", event)

    return resultado.group(1).strip()

In [46]:
# Creating the file
def createCsvPicks(filename, head):
    """Create the file where the data is load"""
    with open(filename, "a", newline="") as file:
        writer = csv.writer(file)
        writer.writerow(head)


def writeFunction(filename, teamA, teamB, header=0):
    """Copy the data to a file"""
    with open(filename, "a", newline="") as file:
        writer = csv.writer(file)
        writer.writerow(teamA)
        writer.writerow(teamB)
        file.close()


def createCsvDict(filename, rows):  # Creation of the file and headers
    """Create the dictionary with the map stats"""
    with open(
        filename, "a", newline=""
    ) as csvfile:  # creation of the file with te headers
        writer = csv.writer(csvfile)
        writer.writerow(rows.keys())
        csvfile.close()


def dictWritter(filename, rows):  # stats from the teams
    """Copy info to the dictionary"""
    with open(filename, "a", newline="") as csvfile:
        writer = csv.writer(csvfile)
        writer.writerows(zip(*rows.values()))
        csvfile.close()


def csvRondas(rows, filename="rndDetail.csv"):
    createCsvDict(filename, rows)

In [47]:
# bkup del cambio en pistol rounds
def pistolRounds(soup, rowInfo, bo=3):
    """Extraction of the pistol rounds"""
    if bo == 3:
        rondas = [x for x in range(1, 67, 13)]
    elif bo == 5:
        rondas = [x for x in range(1, 119, 13)]

    bloques_pistol = soup.find_all("div", class_="vlr-rounds-row-col")
    try:
        for ronda in rondas:
            rondaEvaluada = re.findall(r"rnd-sq(.*)", str(bloques_pistol[ronda]))
            if rondaEvaluada[0] == ' mod-win mod-ct">':
                rowInfo["own_pistol_ct"].append(1)
            elif rondaEvaluada[0] == ' mod-win mod-t">':
                rowInfo["own_pistol_tt"].append(1)
            elif rondaEvaluada[1] == ' mod-win mod-ct">':
                rowInfo["own_pistol_tt"].append(0)
            elif rondaEvaluada[1] == ' mod-win mod-t">':
                rowInfo["own_pistol_ct"].append(0)
            if rondaEvaluada[0] == "rnd-sq" and rondaEvaluada[1] == "rnd-sq":
                rowInfo["own_pistol_ct"].append(0)
                rowInfo["own_pistol_tt"].append(0)
    except:
        pass

In [48]:
def extraerScore(soup, mapasJuntos, rowInfo, bo=3):  # testeo
    """
    Function for scores by side and map played
    what is missing: Map pick owner
    """
    bloqueOT = soup.find_all("div", class_="vlr-rounds-row-col")
    rowOT = {"teamACT": 0, "teamATT": 0, "teamBCT": 0, "teamBTT": 0}
    bloques_equipos = soup.find_all("div", class_="team")
    for map in mapasJuntos:
        rowInfo["map"].append(map)
    for count, bloque in enumerate(bloques_equipos):
        if count % 2 == 0:
            for count, ronda in enumerate(bloqueOT):
                try:
                    valor = int(ronda.find_all("div", class_="rnd-num")[0].text.strip())
                    if valor > 24:
                        rondaEvaluada = re.findall(r"rnd-sq(.*)", str(bloqueOT[count]))
                        if rondaEvaluada[0] == ' mod-win mod-ct">':
                            rowOT["teamACT"] += 1
                        elif rondaEvaluada[0] == ' mod-win mod-t">':
                            rowOT["teamATT"] += 1
                        if rondaEvaluada[1] == ' mod-win mod-ct">':
                            rowOT["teamBCT"] += 1
                        elif rondaEvaluada[1] == ' mod-win mod-t">':
                            rowOT["teamBTT"] += 1

                except:
                    pass
            try:
                equipo = bloque.find("div", class_="team-name").text.strip()
                rowInfo["team"].append(equipo)
                spans = bloque.find_all("span")
                for span in spans:
                    if span["class"][0] == "mod-t":
                        rowInfo["own_score_tt"].append(
                            span.text.strip() + rowOT["teamATT"]
                        )
                    if span["class"][0] == "mod-ct":
                        rowInfo["own_score_ct"].append(
                            span.text.strip() + rowOT["teamACT"]
                        )

            except:
                pass
        if count % 2 == 1:
            try:
                equipo = bloque.find("div", class_="team-name").text.strip()
                rowInfo["rival"].append(equipo)
                spans = bloque.find_all("span")
                for span in spans:
                    if span["class"][0] == "mod-t":
                        rowInfo["rival_score_tt"].append(span.text.strip())
                    if span["class"][0] == "mod-ct":
                        rowInfo["rival_score_ct"].append(span.text.strip())
            except:
                pass
    pistolRounds(soup, rowInfo, bo)
    lenCorrector(rowInfo)
    return rowInfo

In [49]:
def OTextract(soup, rowInfo, bo=3):
    """Extraction of the OT rounds"""
    rowOT = {"teamACT": 0, "teamATT": 0, "teamBCT": 0, "teamBTT": 0}
    bloques_pistol = soup.find_all("div", class_="vlr-rounds-row-col")
    verificador = 0
    mapNumber = 0
    for count, ronda in enumerate(bloques_pistol):

        try:
            valor = int(ronda.find_all("div", class_="rnd-num")[0].text.strip())
            if valor >= verificador:
                verificador = valor
                if valor > 24:
                    rondaEvaluada = re.findall(
                        r"rnd-sq(.*)", str(bloques_pistol[count])
                    )
                    if rondaEvaluada[0] == ' mod-win mod-ct">':
                        rowOT["teamACT"] += 1
                    elif rondaEvaluada[0] == ' mod-win mod-t">':
                        rowOT["teamATT"] += 1
                    if rondaEvaluada[1] == ' mod-win mod-ct">':
                        rowOT["teamBCT"] += 1
                    elif rondaEvaluada[1] == ' mod-win mod-t">':
                        rowOT["teamBTT"] += 1
            else:

                rowInfo["own_score_tt"][mapNumber] = (
                    int(rowInfo["own_score_tt"][mapNumber]) + rowOT["teamATT"]
                )
                rowInfo["own_score_ct"][mapNumber] = (
                    int(rowInfo["own_score_ct"][mapNumber]) + rowOT["teamACT"]
                )
                rowInfo["rival_score_tt"][mapNumber] = (
                    int(rowInfo["rival_score_tt"][mapNumber]) + rowOT["teamBTT"]
                )
                rowInfo["rival_score_ct"][mapNumber] = (
                    int(rowInfo["rival_score_ct"][mapNumber]) + rowOT["teamBCT"]
                )
                mapNumber += 1
                verificador = valor
                rowOT = {"teamACT": 0, "teamATT": 0, "teamBCT": 0, "teamBTT": 0}

        except:
            pass
    rowInfo["own_score_tt"][mapNumber] = (
        int(rowInfo["own_score_tt"][mapNumber]) + rowOT["teamATT"]
    )
    rowInfo["own_score_ct"][mapNumber] = (
        int(rowInfo["own_score_ct"][mapNumber]) + rowOT["teamACT"]
    )
    rowInfo["rival_score_tt"][mapNumber] = (
        int(rowInfo["rival_score_tt"][mapNumber]) + rowOT["teamBTT"]
    )
    rowInfo["rival_score_ct"][mapNumber] = (
        int(rowInfo["rival_score_ct"][mapNumber]) + rowOT["teamBCT"]
    )
    return rowInfo

In [50]:
def rondasCsv(dictRondas, rowInfo, mapNumber, filename="rndDetail.csv"):
    dictFinal = {
        "teamA": [],
        "map": [],
        "side": [],
        "teamB": [],
        "rndA": [],
        "rndB": [],
        "round": [],
        "winCon": [],
        "date": [],
        "map_order": [],
        'event': []
    }
    
    for count, rondaAtk in enumerate(dictRondas["teamATT"]):
        dictFinal["teamA"].append(rowInfo["team"][mapNumber])
        dictFinal["teamB"].append(rowInfo["rival"][mapNumber])
        dictFinal["side"].append("Atk")
        dictFinal["rndA"].append(rondaAtk)
        dictFinal["rndB"].append(dictRondas["teamBCT"][count])
        dictFinal["map"].append(dictRondas["map"][0])
        dictFinal["round"].append(dictRondas["ratk"][count])
        dictFinal["winCon"].append(dictRondas["winConAtk"][count])
        dictFinal["date"].append(dictRondas["date"][count])
        dictFinal["map_order"].append(dictRondas["map_order"][0])
        dictFinal["event"].append(dictRondas["event"][count])

    for count, rondaDef in enumerate(dictRondas["teamACT"]):
        dictFinal["teamA"].append(rowInfo["team"][mapNumber])
        dictFinal["teamB"].append(rowInfo["rival"][mapNumber])
        dictFinal["side"].append("Def")
        dictFinal["rndA"].append(rondaDef)
        dictFinal["rndB"].append(dictRondas["teamBTT"][count])
        dictFinal["map"].append(dictRondas["map"][0])
        dictFinal["round"].append(dictRondas["rdef"][count])
        dictFinal["winCon"].append(dictRondas["winConDef"][count])
        dictFinal["date"].append(dictRondas["date"][count])
        dictFinal["map_order"].append(dictRondas["map_order"][0])
        dictFinal["event"].append(dictRondas["event"][count])


    dictWritter(filename, dictFinal)
    return dictFinal

In [51]:
# No me gusta lo de los diccionarios y como tengo que manualmente poner las rows
def rondaDetalle(soup, rowInfo, mapas, filename="rndDetail.csv"):
    rondas = {
        "map": [],
        "teamACT": [],
        "teamATT": [],
        "teamBCT": [],
        "teamBTT": [],
        "ratk": [],
        "rdef": [],
        "winConAtk": [],
        "winConDef": [],
        "date": [],
        "map_order": [],
        'event':[]
    }
    
    bloques = soup.find_all("div", class_="vlr-rounds-row-col")
    verificador = 0
    mapNumber = 0
    rondas["map"].append(mapas[mapNumber])
    rondas["map_order"].append(mapNumber)
    event = event_name(soup)
    for count, ronda in enumerate(bloques):
        try:
            valor = int(ronda.find_all("div", class_="rnd-num")[0].text.strip())
            imgUrl = str(ronda.find_all("img")[0])
            vict = imgUrl[0:-3].split("/")[-1].rstrip(".webp")
            fecha = soup.find_all("div", class_="moment-tz-convert")[0].get(
                "data-utc-ts"
            )
            rondas["date"].append(fecha)

           

            rondas['event'].append(event)


            if valor >= verificador:
                verificador = valor
                rondaEvaluada = re.findall(r"rnd-sq(.*)", str(bloques[count]))
                if rondaEvaluada[0] == ' mod-win mod-ct">':
                    rondas["teamACT"].append(1)
                    rondas["teamBTT"].append(0)
                    rondas["rdef"].append(valor)
                    rondas["winConDef"].append(vict)
                elif rondaEvaluada[0] == ' mod-win mod-t">':
                    rondas["teamATT"].append(1)
                    rondas["teamBCT"].append(0)
                    rondas["ratk"].append(valor)
                    rondas["winConAtk"].append(vict)
                if rondaEvaluada[1] == ' mod-win mod-ct">':
                    rondas["teamBCT"].append(1)
                    rondas["teamATT"].append(0)
                    rondas["ratk"].append(valor)
                    rondas["winConAtk"].append(vict)
                elif rondaEvaluada[1] == ' mod-win mod-t">':
                    rondas["teamBTT"].append(1)
                    rondas["teamACT"].append(0)
                    rondas["rdef"].append(valor)
                    rondas["winConDef"].append(vict)

            else:
                mapNumber += 1
                verificador = valor
                rondasCsv(rondas, rowInfo, mapNumber, filename)
                rondas = {
                    "map": [],
                    "teamACT": [],
                    "teamATT": [],
                    "teamBCT": [],
                    "teamBTT": [],
                    "ratk": [],
                    "rdef": [],
                    "winConAtk": [],
                    "winConDef": [],
                    "date": [],
                    "map_order": [],
                    'event': [],
                }
                rondas["map_order"].append(mapNumber)
                rondas["map"].append(mapas[mapNumber])
                rondaEvaluada = re.findall(r"rnd-sq(.*)", str(bloques[count]))
                imgUrl = str(ronda.find_all("img")[0])
                vict = imgUrl[0:-3].split("/")[-1].rstrip(".webp")
                fecha = soup.find_all("div", class_="moment-tz-convert")[0].get(
                    "data-utc-ts"
                )
                rondas["date"].append(fecha)
                rondas['event'].append(event)

                if rondaEvaluada[0] == ' mod-win mod-ct">':
                    rondas["teamACT"].append(1)
                    rondas["teamBTT"].append(0)
                    rondas["rdef"].append(valor)
                    rondas["winConDef"].append(vict)
                elif rondaEvaluada[0] == ' mod-win mod-t">':
                    rondas["teamATT"].append(1)
                    rondas["teamBCT"].append(0)
                    rondas["ratk"].append(valor)
                    rondas["winConAtk"].append(vict)
                if rondaEvaluada[1] == ' mod-win mod-ct">':
                    rondas["teamBCT"].append(1)
                    rondas["teamATT"].append(0)
                    rondas["ratk"].append(valor)
                    rondas["winConAtk"].append(vict)
                elif rondaEvaluada[1] == ' mod-win mod-t">':
                    rondas["teamBTT"].append(1)
                    rondas["teamACT"].append(0)
                    rondas["rdef"].append(valor)
                    rondas["winConDef"].append(vict)
        except:
            pass
    
    rondasCsv(rondas, rowInfo, mapNumber, filename)
    return rondas

In [52]:
def pistolRounds(soup, rowInfo, bo=3):
    """Extraction of the pistol rounds"""
    bloques_pistol = soup.find_all("div", class_="vlr-rounds-row-col")
    for count, ronda in enumerate(bloques_pistol):
        try:
            valor = int(ronda.find_all("div", class_="rnd-num")[0].text.strip())
            if valor == 1 or valor == 13:
                rondaEvaluada = re.findall(r"rnd-sq(.*)", str(bloques_pistol[count]))
                if rondaEvaluada[0] == ' mod-win mod-ct">':
                    rowInfo["own_pistol_ct"].append(1)
                elif rondaEvaluada[0] == ' mod-win mod-t">':
                    rowInfo["own_pistol_tt"].append(1)
                elif rondaEvaluada[1] == ' mod-win mod-ct">':
                    rowInfo["own_pistol_tt"].append(0)
                elif rondaEvaluada[1] == ' mod-win mod-t">':
                    rowInfo["own_pistol_ct"].append(0)
                if rondaEvaluada[0] == "rnd-sq" and rondaEvaluada[1] == "rnd-sq":
                    rowInfo["own_pistol_ct"].append(0)
                    rowInfo["own_pistol_tt"].append(0)
        except:
            pass


def extraerScore(soup, mapasJuntos, rowInfo, bo=3): #Todo este bloque de codigo esta raro, entre con el rowinfo pero no lo defino dentro, no se que tiene 
    """
    Function for scores by side and map played
    what is missing: Map pick owner
    """

    bloques_equipos = soup.find_all("div", class_="team")
    for map in mapasJuntos:
        rowInfo["map"].append(map)
    for count, bloque in enumerate(bloques_equipos):
        if count % 2 == 0:
            try:
                equipo = bloque.find("div", class_="team-name").text.strip()
                rowInfo["team"].append(equipo)
                spans = bloque.find_all("span")
                for span in spans:
                    if span["class"][0] == "mod-t":
                        rowInfo["own_score_tt"].append(span.text.strip())
                    if span["class"][0] == "mod-ct":
                        rowInfo["own_score_ct"].append(span.text.strip())
            except:
                pass
        if count % 2 == 1:
            try:
                equipo = bloque.find("div", class_="team-name").text.strip()
                rowInfo["rival"].append(equipo)
                spans = bloque.find_all("span")
                for span in spans:
                    if span["class"][0] == "mod-t":
                        rowInfo["rival_score_tt"].append(span.text.strip())
                    if span["class"][0] == "mod-ct":
                        rowInfo["rival_score_ct"].append(span.text.strip())
            except:
                pass
    pistolRounds(soup, rowInfo, bo)
    lenCorrector(rowInfo)
    OTextract(soup, rowInfo, bo)

    #Agregue esto esto con el extend es raro, deberia checkearlo
    fecha = soup.find_all("div", class_="moment-tz-convert")[0].get(
                "data-utc-ts"
        )
    rowInfo['date'].extend(repeat(fecha, len(rowInfo['map'])))

    event = event_name(soup)

    rowInfo['event'].extend(repeat(event, len(rowInfo['map'])))

    #hasta aca
    # testeo de agregar el detalle de rondas. Parece estar funcionando
    rondaDetalle(soup, rowInfo, mapasJuntos)
    return rowInfo


# Extract total scores from a match
def scoreTotal(soup):
    """Total score (not used right now)"""
    score = soup.find("div", class_="match-header-vs-score")
    spanScores = score.find_all("span")
    equipoIzq = int(spanScores[0].text.strip())
    equipoDer = int(spanScores[2].text.strip())
    return equipoIzq + equipoDer

In [53]:
def pickBans(soup):
    """Picks and bans extractor"""
    try:
        picksBans = soup.find(
            "div", {"class": "match-header-note"}
        ).get_text()  # Extraccion del texto
        picksBans = picksBans.strip()  # Remover los separados y espacios
        picksBans = picksBans.split(";")
    except:
        picksBans = ""
    return picksBans

In [54]:
def pickBansExtractor(soup, bo=3, filename="PicksBans.csv"):
    picks_bans = pickBans(soup)
    header = [
        "team",
        "rival",
        "team_1_select_1",
        "team_2_select_1",
        "team_1_select_2",
        "team_2_select_2",
        "team_1_select_3",
        "team_2_select_3",
        "decider",
        "order",
        "bo",
        'date',
        'event',
    ]
    dict_picks_bans = {"team_A": [], "team_B": []}
    team_a = picks_bans[0].split()[0]
    team_b = picks_bans[3].split()[0]
    dict_picks_bans["team_A"].append(team_a)
    dict_picks_bans["team_A"].append(team_b)
    dict_picks_bans["team_B"].append(team_b)
    dict_picks_bans["team_B"].append(team_a)

    for element in picks_bans:
        list_element = element.split()
        if len(list_element) == 3:
            dict_picks_bans["team_A"].append(list_element[-1])
        if len(list_element) == 2:
            dict_picks_bans["team_A"].append(list_element[0])

    order_team_B = [1, 0, 3, 2, 5, 4, 6]
    maps_teamA = dict_picks_bans["team_A"][2:]

    ordenado = [maps_teamA[i] for i in order_team_B]

    for map in ordenado:
        dict_picks_bans["team_B"].append(map)

    dict_picks_bans["team_A"].append("A")
    dict_picks_bans["team_B"].append("B")

    dict_picks_bans["team_A"].append(bo)
    dict_picks_bans["team_B"].append(bo)

    #ACA
        #Agregue esto
    fecha = soup.find_all("div", class_="moment-tz-convert")[0].get(
                "data-utc-ts"
        )
    dict_picks_bans["team_A"].append(fecha)
    dict_picks_bans["team_B"].append(fecha)

    event = event_name(soup)

    dict_picks_bans["team_A"].append(event)
    dict_picks_bans["team_B"].append(event)
    
    #hasta aca
    return header, dict_picks_bans["team_A"], dict_picks_bans["team_B"]

In [55]:
def runExtractor(soup, bo=3, filename="PicksBans.csv"):
    header, teamA, teamB = pickBansExtractor(soup, bo, filename)
    if bo == 3:
        mapas_juntos = [teamA[4], teamB[4], teamA[8]]
    if bo == 5:
        mapas_juntos = [teamA[4], teamA[5], teamA[6], teamA[7], teamA[8]]

    return header, teamA, teamB, mapas_juntos

In [56]:
def linkExtractor(url):
    """With the url from matches all in vlr, create a list of links with
    all the matches.
    """
    page = urlopen(url)
    html = page.read().decode("utf-8")
    soup = BeautifulSoup(html, "html.parser")
    tempLink = []
    urlLinkExtract = []
    for a in soup.find_all("a", href=True):
        tempLink.append(a["href"])
        filtered_links = [link for link in tempLink if re.match(r"^/\d+", link)]

    for cleanLink in filtered_links:
        urlLinkExtract.append("https://www.vlr.gg" + cleanLink)

    return urlLinkExtract

In [57]:
def team_df(soup,team_dict):
    event = event_name(soup)

    string = soup.find('title').get_text(strip=True)
    pattern = r"^(.*?) vs\. (.*?) \|"

    result = re.findall(pattern, string)
    teams = list(result[0]) if result else []

    if teams[0] in team_dict["team_name"]:
        pass
    else: 
        teamA_tricode = soup.find_all('div',{'class':'team'})[2].get_text(strip=True)
        
        team_dict['team_name'].append(teams[0])
        team_dict['tricode'].append(teamA_tricode)
        team_dict['event'].append(event)

    if teams[1] in team_dict["team_name"]:
        pass
    else: 
        teamB_tricode = soup.find_all('div',{'class':'team'})[3].get_text(strip=True)
        
        team_dict['team_name'].append(teams[1])
        team_dict['tricode'].append(teamB_tricode)
        team_dict['event'].append(event)
    return team_dict

In [58]:
def serialExtractor(soup, bo, fileNames):
    """Serial extractor working with url"""
    # page = urlopen(url)
    # html = page.read().decode("utf-8")
    # soup = BeautifulSoup(html, "html.parser")
    rows = {
        "team": [],
        "rival": [],
        "map": [],
        "own_pistol_tt": [],
        "own_pistol_ct": [],
        "own_score_tt": [],
        "rival_score_tt": [],
        "own_score_ct": [],
        "rival_score_ct": [],
        'date': [],
        'event': []
    }

    header, teamA, teamB, mapasJuntos = runExtractor(soup, bo)

    writeFunction(fileNames[0], teamA, teamB)

    rowInfo = extraerScore(soup, mapasJuntos, rows, bo)
    
    dictWritter(fileNames[1], rowInfo)
    time.sleep(random.randint(1, 3))  # Sleep de 1 a 3 sec para no floodear la pagina

In [59]:
def urlStats(url):
    page = urlopen(url)
    html = page.read().decode("utf-8")
    soup = BeautifulSoup(html, "html.parser")
    return soup

In [60]:
def verificador(url):
    """Verificador de info"""
    page = urlopen(url)
    html = page.read().decode("utf-8")
    soup = BeautifulSoup(html, "html.parser")
    mejorDe = 0
    estado = 0
    estados = soup.find_all("div", class_="match-header-vs-note")
    if estados[0].text.strip() == "final":
        estado = "final"
        if estados[1].text.strip() == "Bo3":
            mejorDe = 3
        elif estados[1].text.strip() == "Bo5":
            mejorDe = 5
    return mejorDe, estado

In [61]:
# urlLinks = "https://www.vlr.gg/event/matches/2274/champions-tour-2025-americas-kickoff/?series_id=4405" #Amercias

# urlLinks = "https://www.vlr.gg/event/matches/2276/champions-tour-2025-emea-kickoff/?series_id=all" #Emea

# urlLinks = "https://www.vlr.gg/event/matches/2277/champions-tour-2025-pacific-kickoff/?series_id=all" #Pacific

# urlLinks = "https://www.vlr.gg/event/matches/2275/champions-tour-2025-china-kickoff/?series_id=all" #China

urlLinks = "https://www.vlr.gg/event/matches/2281/champions-tour-2025-masters-bangkok/?series_id=all"  # Masters 1

toExtract = linkExtractor(urlLinks)

In [62]:
# Preparación del bloque para correr el for
fileName = "picks.csv" #amer (saque eso)
fileNameDict = "Statsamer1.csv"
fileNamebo5 = "bo5amer1.csv"
files = [fileName, fileNameDict, fileNamebo5]
header = [
    "team",
    "rival",
    "team_1_select_1",
    "team_2_select_1",
    "team_1_select_2",
    "team_2_select_2",
    "team_1_select_3",
    "team_2_select_3",
    "decider",
    "order",
    "bo",
    'date',
    'event',
]
rows = {
    "team": [],
    "rival": [],
    "map": [],
    "own_pistol_tt": [],
    "own_pistol_ct": [],
    "own_score_tt": [],
    "rival_score_tt": [],
    "own_score_ct": [],
    "rival_score_ct": [],
    'date':[],
    'event':[],
}
createCsvPicks(fileName, header)
createCsvDict(fileNameDict, rows)

# Create csv rondas
filenameRondas = "rndDetail.csv"
dictFinal = {
    "teamA": [],
    "map": [],
    "side": [],
    "teamB": [],
    "rndA": [],
    "rndB": [],
    "round": [],
    "winCon": [],
    "date": [],
    "map_order": [],
    'event': [],
}
createCsvDict(filenameRondas, dictFinal)

team_dict = {'team_name':[],
            'tricode':[],
            'event':[]}

In [63]:
# Aca arranca el for (probablemente se pueda optimizar y no es necesario correr en bucle limpiar las rows y pasar al csv)
# agregar que pasa cuando no se saben los picks y bans.
# Este For es Picks, stats por equipo y stats por ronda
for dire in toExtract:
    mejorDe, estado = verificador(dire)
    if estado == "final" and (mejorDe == 3 or mejorDe == 5):
        soup = urlStats(dire)
        team_df(soup,team_dict)
        serialExtractor(soup, mejorDe, files)
    elif estado != "final":
        print("Match no finalizado: " + dire)
    elif mejorDe == 5:
        print("Match mejor de 5: " + dire)

df_teams = pd.DataFrame(team_dict)
df_teams.to_csv('teams.csv',index=False) #ojo index primera finla 

In [64]:
playerStats = {
    'team':[],
    "player": [],
    "agent": [],
    "ratingBoth": [],
    "ratingT": [],
    "rating-ct": [],
    "acsBoth": [],
    "acsT": [],
    "acsCT": [],
    "killsBoth": [],
    "killsT": [],
    "killsCT": [],
    "deadBoth": [],
    "deadT": [],
    "deadCT": [],
    "assistsBoth": [],
    "assistsT": [],
    "assistsCT": [],
    "k-dBoth": [],
    "k-dT": [],
    "k-dCT": [],
    "kastBoth": [],
    "kastT": [],
    "kastCT": [],  
    "adrBoth": [],
    "adrT": [],
    "adrCT": [],
    "hsBoth": [],
    "hsT": [],
    "hsCT": [],
    "fkBoth": [],
    "fkT": [],
    "fkCT": [],
    "fdBoth": [],
    "fdT": [],
    "fdCT": [],
    "fk-fdBoth": [],
    "fk-fdT": [],
    "fk-fdCT": [],
    "map": [],
    "date": [],
    'event':[]
}


def PlayerStatsExtractror(soup, playerStats):
    extractPlayer = soup.find_all(
        "td", class_="mod-player"
    )  # extracción de los nombres de los players (ojo que el CREO que el primero es globlar y despues por mapa)
    fecha = soup.find_all("div", class_="moment-tz-convert")[0].get(
                "data-utc-ts"
        )
    event = event_name(soup)
    

    # Extract players names
    for name in extractPlayer:
        player = name.find(class_="text-of").get_text()
        player = player.strip().split(";")
        team_name = name.find(class_='ge-text-light').get_text(strip=True)

        playerStats["player"].append(player[0])
        playerStats['team'].append(team_name)
        playerStats['date'].append(fecha)
        playerStats['event'].append(event)
    # Extract agent played
    extractAgent = soup.find_all("td", class_="mod-agents")
    for agent in extractAgent:
        agent = agent.find("img").get("title")
        playerStats["agent"].append(agent)

    # Extract VLR rating, ACS, Kills both sides
    extratRating = soup.find_all("td", class_="mod-stat")
    contador = 0
    for mod in extratRating:
        try:
            if contador == 0:
                mod = mod.find(class_="side mod-side mod-both").get_text()
                playerStats["ratingBoth"].append(float(mod))
                contador += 1
            if contador == 1:
                mod = mod.find(class_="side mod-side mod-both").get_text()
                playerStats["acsBoth"].append(float(mod))
                contador += 1
            if contador == 2:
                mod = mod.find(class_="side mod-side mod-both").get_text()
                playerStats["killsBoth"].append(float(mod))
                contador = 0
        except:
            pass

    # Extract VLR rating, ACS, Kills attack side
    contador = 0
    for mod in extratRating:
        try:
            if contador == 0:
                mod = mod.find(class_="side mod-side mod-t").get_text()
                playerStats["ratingT"].append(float(mod))
                contador += 1
            if contador == 1:
                mod = mod.find(class_="side mod-side mod-t").get_text()
                playerStats["acsT"].append(float(mod))
                contador += 1
            if contador == 2:
                mod = mod.find(class_="side mod-side mod-t").get_text()
                playerStats["killsT"].append(float(mod))
                contador = 0
        except:
            pass

    # Extract VLR rating, ACS, Kills defend side
    contador = 0
    for mod in extratRating:
        try:
            if contador == 0:
                mod = mod.find(class_="side mod-side mod-ct").get_text()
                playerStats["rating-ct"].append(float(mod))
                contador += 1
            if contador == 1:
                mod = mod.find(class_="side mod-side mod-ct").get_text()
                playerStats["acsCT"].append(float(mod))
                contador += 1
            if contador == 2:
                mod = mod.find(class_="side mod-side mod-ct").get_text()
                playerStats["killsCT"].append(float(mod))
                contador = 0
        except:
            pass

    # Stats Both Kills, assits,k-d,kast,adr,hs,fk,fd,fk-fd
    extractText = soup.find_all("td", class_="mod-stat")
    contador = 0
    for mod in extractText:
        try:
            valorTemp = mod.find(
                True,
                {
                    "class": [
                        "side mod-both",
                        "side mod-both mod-positive",
                        "side mod-both mod-negative",
                    ]
                },
            ).get_text()
            if contador == 0:
                playerStats["deadBoth"].append(float(valorTemp))
                contador += 1
            elif contador == 1:
                playerStats["assistsBoth"].append(float(valorTemp))
                contador += 1
            elif contador == 2:
                playerStats["k-dBoth"].append(float(valorTemp))
                contador += 1
            elif contador == 3:
                playerStats["kastBoth"].append(valorTemp)
                contador += 1
            elif contador == 4:
                playerStats["adrBoth"].append(float(valorTemp))
                contador += 1
            elif contador == 5:
                playerStats["hsBoth"].append(valorTemp)
                contador += 1
            elif contador == 6:
                playerStats["fkBoth"].append(float(valorTemp))
                contador += 1
            elif contador == 7:
                playerStats["fdBoth"].append(float(valorTemp))
                contador += 1
            elif contador == 8:
                playerStats["fk-fdBoth"].append(float(valorTemp))
                contador = 0
        except:
            pass

    contador = 0
    for mod in extractText:
        try:
            valorTemp = mod.find(
                True,
                {
                    "class": [
                        "side mod-t",
                        "side mod-t mod-positive",
                        "side mod-t mod-negative",
                    ]
                },
            ).get_text()
            if contador == 0:
                playerStats["deadT"].append(float(valorTemp))
                contador += 1
            elif contador == 1:
                playerStats["assistsT"].append(float(valorTemp))
                contador += 1
            elif contador == 2:
                playerStats["k-dT"].append(float(valorTemp))
                contador += 1
            elif contador == 3:
                playerStats["kastT"].append(valorTemp)
                contador += 1
            elif contador == 4:
                playerStats["adrT"].append(float(valorTemp))
                contador += 1
            elif contador == 5:
                playerStats["hsT"].append(valorTemp)
                contador += 1
            elif contador == 6:
                playerStats["fkT"].append(float(valorTemp))
                contador += 1
            elif contador == 7:
                playerStats["fdT"].append(float(valorTemp))
                contador += 1
            elif contador == 8:
                playerStats["fk-fdT"].append(float(valorTemp))
                contador = 0
        except:
            pass

    contador = 0
    for mod in extractText:
        try:
            valorTemp = mod.find(
                True,
                {
                    "class": [
                        "side mod-ct",
                        "side mod-ct mod-positive",
                        "side mod-ct mod-negative",
                    ]
                },
            ).get_text()
            if contador == 0:
                playerStats["deadCT"].append(float(valorTemp))
                contador += 1
            elif contador == 1:
                playerStats["assistsCT"].append(float(valorTemp))
                contador += 1
            elif contador == 2:
                playerStats["k-dCT"].append(float(valorTemp))
                contador += 1
            elif contador == 3:
                playerStats["kastCT"].append(valorTemp)
                contador += 1
            elif contador == 4:
                playerStats["adrCT"].append(float(valorTemp))
                contador += 1
            elif contador == 5:
                playerStats["hsCT"].append(valorTemp)
                contador += 1
            elif contador == 6:
                playerStats["fkCT"].append(float(valorTemp))
                contador += 1
            elif contador == 7:
                playerStats["fdCT"].append(float(valorTemp))
                contador += 1
            elif contador == 8:
                playerStats["fk-fdCT"].append(float(valorTemp))
                contador = 0
        except:
            pass
    return playerStats

In [65]:
def addMap(soup):
    picksBans = soup.find(
        "div", {"class": "match-header-note"}
    ).get_text()  # Extraccion del texto
    picksBans = picksBans.strip()  # Remover los separados y espacios
    picksBans = picksBans.split(";")
    tempMaps = []
    pick_pattern = re.compile(r"pick (\w+)")

    for item in picksBans:
        match = pick_pattern.search(item)
        if match:
            tempMaps.append(match.group(1))

    pick_pattern = re.compile(r"(\w+) remains")

    for item in picksBans:
        match = pick_pattern.search(item)
        if match:
            tempMaps.append(match.group(1))

    tempMaps.insert(1, "All maps")
    # Calculo del largo de la cadena de mapas (sacar mapa no jugado)
    score = soup.find("div", {"class": "js-spoiler"}).get_text()
    score = score.replace("\n", "").replace(
        "\t", ""
    )  # Remover los separados y espacios

    score = score.strip()
    totalScore = float(score[0]) + float(score[-1]) + 1
    finalMaps = []
    for map in tempMaps[: int(totalScore)]:
        finalMaps.extend(repeat(map, 10))

    return finalMaps

In [66]:
def add_map_performance_page(soup):    
    picksBans = soup.find(
        "div", {"class": "match-header-note"}
    ).get_text()  # Extraccion del texto
    picksBans = picksBans.strip()  # Remover los separados y espacios
    picksBans = picksBans.split(";")
    tempMaps = []
    pick_pattern = re.compile(r"pick (\w+)")

    for item in picksBans:
        match = pick_pattern.search(item)
        if match:
            tempMaps.append(match.group(1))

    pick_pattern = re.compile(r"(\w+) remains")

    for item in picksBans:
        match = pick_pattern.search(item)
        if match:
            tempMaps.append(match.group(1))

    tempMaps.insert(0, "All maps")
    # Calculo del largo de la cadena de mapas (sacar mapa no jugado)
    score = soup.find("div", {"class": "js-spoiler"}).get_text()
    score = score.replace("\n", "").replace(
        "\t", ""
    )  # Remover los separados y espacios

    score = score.strip()
    totalScore = float(score[0]) + float(score[-1]) + 1
    finalMaps = []
    final_date = []
    event_list = []
    event = event_name(soup)
    for map in tempMaps[: int(totalScore)]: #sacar de adentro el calculo e fecha y evento
        finalMaps.extend(repeat(map, 10))    
        fecha = soup.find_all("div", class_="moment-tz-convert")[0].get("data-utc-ts")
        final_date.extend(repeat(fecha, 10))
        event_list.extend(repeat(event,10))

    return finalMaps, final_date,event_list

In [67]:
# Extractor de los stats de los players vuelvo a hacer call de a la url, esto deberia ser optimizado
# Separado de lo otro, tengo que encontrar la manera de no recallear cosas
# def urlStats(url):
#     page = urlopen(url)
#     html = page.read().decode("utf-8")
#     soup = BeautifulSoup(html, "html.parser")
#     return soup


playerStats = {
    'team': [],
    "player": [],
    "agent": [],
    "ratingBoth": [],
    "ratingT": [],
    "rating-ct": [],
    "acsBoth": [],
    "acsT": [],
    "acsCT": [],
    "killsBoth": [],
    "killsT": [],
    "killsCT": [],
    "deadBoth": [],
    "deadT": [],
    "deadCT": [],
    "assistsBoth": [],
    "assistsT": [],
    "assistsCT": [],
    "k-dBoth": [],
    "k-dT": [],
    "k-dCT": [],
    "kastBoth": [],
    "kastT": [],
    "kastCT": [], 
    "adrBoth": [],
    "adrT": [],
    "adrCT": [],
    "hsBoth": [],
    "hsT": [],
    "hsCT": [],
    "fkBoth": [],
    "fkT": [],
    "fkCT": [],
    "fdBoth": [],
    "fdT": [],
    "fdCT": [],
    "fk-fdBoth": [],
    "fk-fdT": [],
    "fk-fdCT": [],
    "map": [],
    'date':[],
    'event':[],
}

pickedMaps = []

for dire in toExtract:
    sopa = urlStats(dire)
    mejorDe, estado = verificador(dire)
    if estado == "final" and (mejorDe == 3 or mejorDe == 5):
        statsDict = PlayerStatsExtractror(sopa, playerStats)
        pickedMaps = addMap(sopa)
        playerStats["map"].extend(pickedMaps)
    elif estado != "final":
        print("Match no finalizado: " + dire)

listaCategoraias = list(playerStats)
df = pd.DataFrame(playerStats)

df.to_csv("playerstats.csv", index=False)

In [68]:
#Ordenar un poco mejor el codigo 
performance_dict = {
    "player": [],
    'team': [],
    "2K": [],
    "3K": [],
    "4K": [],
    "5K": [],
    "1v1": [],
    "1v2": [],
    "1v3": [],
    "1v4": [],
    "1v5": [],
    "ECON": [],
    "PL": [],
    "DE": [],
    "map": [],
    "date": [],
    'event':[]
}

for dire in toExtract:
    dire_perfo = dire+'/?game=all&tab=performance'
    sopa = urlStats(dire_perfo)
    soup_pickedMap = urlStats(dire)
    mejorDe, estado = verificador(dire)
    if estado == "final" and (mejorDe == 3 or mejorDe == 5):
        # pickedMap = add_map_performance_page(soup_pickedMap)
        
        get_games_id = sopa.find_all("div", {"class": "vm-stats-game"})
        game_ids = [
            div.get("data-game-id") for div in get_games_id if div.has_attr("data-game-id")
        ]


        for id in game_ids:
            div = sopa.find("div", {"class": "vm-stats-game", "data-game-id": id})
            test_div = div.find_all("tr")[1:]
            pre_process = []
            for element in test_div:
                if len(element) > 13:
                    pre_process.append(element)

            filas = pre_process[1:]
            # Lista para almacenar los datos
            jugadores = []

            for fila in filas:
                celdas = fila.find_all("td")

                if len(celdas) > 0:  # Para asegurarnos de que hay datos válidos en la fila
                    # Extraer nombre del jugador
                    jugador_div = celdas[0].find("div").find_all("div")[0]
                    
                    nombre_jugador = jugador_div.get_text().split() # Primera línea es el nombre
                    
                    def extraer_numero(texto):
                        match = re.match(r"^\d+", texto)
                        return int(match.group()) if match else 0

                    performance_dict["player"].append(nombre_jugador[0])
                    performance_dict["team"].append(nombre_jugador[1])
                    performance_dict["2K"].append(
                        extraer_numero(celdas[2].get_text(strip=True))
                    )
                    performance_dict["3K"].append(
                        extraer_numero(celdas[3].get_text(strip=True))
                    ),
                    performance_dict["4K"].append(
                        extraer_numero(celdas[4].get_text(strip=True))
                    ),
                    performance_dict["5K"].append(
                        extraer_numero(celdas[5].get_text(strip=True))
                    ),
                    performance_dict["1v1"].append(
                        extraer_numero(celdas[6].get_text(strip=True))
                    ),
                    performance_dict["1v2"].append(
                        extraer_numero(celdas[7].get_text(strip=True))
                    ),
                    performance_dict["1v3"].append(
                        extraer_numero(celdas[8].get_text(strip=True))
                    ),
                    performance_dict["1v4"].append(
                        extraer_numero(celdas[9].get_text(strip=True))
                    ),
                    performance_dict["1v5"].append(
                        extraer_numero(celdas[10].get_text(strip=True))
                    ),
                    performance_dict["ECON"].append(
                        extraer_numero(celdas[11].get_text(strip=True))
                    ),
                    performance_dict["PL"].append(
                        extraer_numero(celdas[12].get_text(strip=True))
                    ),
                    performance_dict["DE"].append(
                        extraer_numero(celdas[13].get_text(strip=True))
                    )
        final_map, final_date,final_event = add_map_performance_page(soup_pickedMap)
        performance_dict["map"].extend(final_map)
        performance_dict["date"].extend(final_date)
        performance_dict["event"].extend(final_event) 

    elif estado != "final":
        print("Match no finalizado: " + dire)

csv_file = 'performance.csv'

with open(csv_file, mode='w', newline='') as file:
    writer = csv.writer(file)

    headers = performance_dict.keys()
    writer.writerow(headers)
    
    rows = zip(*performance_dict.values())
    for row in rows:
        writer.writerow(row)


In [69]:
#Fix sopa / soup como nomenclatura
dict_economia = {
    "team_a": [],
    "team_b": [],
    "team_a_economy": [],
    "team_b_economy": [],
    "round": [],
    "map": [],
    "date": [],
    'event': [],
}
for dire in toExtract:
    economy_page = dire + "/?game=all&tab=economy"

    sopa = urlStats(economy_page)

    if estado == "final" and (mejorDe == 3 or mejorDe == 5):

        mapas = pickBans(sopa)
        mapa_elegido = []

        for seleccion in mapas:
            if seleccion.find("pick") != -1:
                mapa_elegido.append(seleccion.split()[2])

        if seleccion.find("remains") != -1:
            mapa_elegido.append(seleccion.split()[0])
        get_games_id = sopa.find_all("div", {"class": "vm-stats-game"})
        game_ids = [
            div.get("data-game-id")
            for div in get_games_id
            if div.has_attr("data-game-id")
        ]

        for value, id in enumerate(game_ids[:-1]):

            div = sopa.find("div", {"class": "vm-stats-game", "data-game-id": id})
            test_div = div.find_all("tr")[1:]

            teams = []
            round = 0
            fecha = sopa.find_all("div", class_="moment-tz-convert")[0].get(
                "data-utc-ts"
            )
            event = event_name(sopa)

            for fila in test_div:
                celdas = fila.find_all("td")
                if len(teams) < 2:
                    teams.append(
                        celdas[0]
                        .find_all("div", {"class": "team"})[0]
                        .get_text(strip=True)
                    )
                for bank in celdas:
                    team_bank = bank.find_all("div", {"class": "bank"})
                    if len(team_bank) > 0:
                        round += 1
                        dict_economia["team_a"].append(teams[0])
                        dict_economia["team_b"].append(teams[1])
                        dict_economia["team_a_economy"].append(
                            team_bank[0].get_text(strip=True)
                        )
                        dict_economia["team_b_economy"].append(
                            team_bank[1].get_text(strip=True)
                        )
                        dict_economia["round"].append(round)
                        dict_economia["map"].append(mapa_elegido[value])
                        dict_economia["date"].append(fecha)
                        dict_economia['event'].append(event)

csv_file = "economy.csv"

with open(csv_file, mode="w", newline="") as file:
    writer = csv.writer(file)
    headers = dict_economia.keys()
    writer.writerow(headers)
    rows = zip(*dict_economia.values())
    for row in rows:
        writer.writerow(row)

In [70]:
def convert_k(valor):
    if 'k' in valor:
        if '.' in valor:
            return int(float(valor.replace('k', '')) * 1000)
        else:
            return int(valor.replace('k', '')) * 1000
    return int(valor)

In [7]:
df_teams = pd.read_csv('teams.csv')
tricode_to_fullname = dict(zip(df_teams['tricode'], df_teams['team_name']))


In [ ]:
#Pipeline for fixing csv differences (for now, team names) OJO CON EL MAP QUE ROMPE

tricode_to_fullname = dict(zip(df_teams['tricode'], df_teams['team_name']))

#Changing economy 
df_economy = pd.read_csv('economy.csv')

df_economy['team_a'] = df_economy['team_a'].map(tricode_to_fullname)
df_economy['team_b'] = df_economy['team_b'].map(tricode_to_fullname)
df_economy['team_a_economy'] = df_economy['team_a_economy'].apply(convert_k)
df_economy['team_b_economy'] = df_economy['team_b_economy'].apply(convert_k)

df_economy.to_csv("economy.csv", index=False)
#Changing performance

df_performance = pd.read_csv('performance.csv')

df_performance['team'] = df_performance['team'].map(tricode_to_fullname)
df_performance.to_csv("performance.csv")
#Changing picksamer

df_picks = pd.read_csv('picks.csv')

df_picks['team'] = df_picks['team'].map(tricode_to_fullname)
df_picks['rival'] = df_picks['rival'].map(tricode_to_fullname)
df_picks.to_csv("picks.csv", index=False)
#changing playerstats

df_player_stats = pd.read_csv('playerstats.csv')

df_player_stats['team'] = df_player_stats['team'].map(tricode_to_fullname)
df_player_stats.to_csv("playerstats.csv")


In [ ]:
#Preprocess for powerBi

df_economy = pd.read_csv('economy.csv')
df_round_detail = pd.read_csv('rndDetail.csv')

df_full_round_detail = pd.merge(left=df_round_detail, right=df_economy, left_on=['date','map','round','event','teamA','teamB'],right_on=['date','map','round','event','team_a','team_b'])

columns_to_df = ['teamA','teamB','map','side','round','team_a_economy','team_b_economy','rndA','rndB','winCon','map_order','date','event']

df_full_round_to_export = df_full_round_detail[columns_to_df]

key_columns = ['event','date','map']
df_full_round_to_export['powerBikey'] = df_full_round_to_export[key_columns].astype(str).agg('_'.join,axis=1)

df_full_round_to_export.to_csv('round_detail_merged.csv')

#remove all map 

df_player_stats = pd.read_csv('playerstats.csv')
df_performance = pd.read_csv('performance.csv')

columns_peformance_df = []
columns_for_merge = ['date','map','event','player','team']

df_performance_merged = pd.merge(left=df_player_stats,right=df_performance,left_on=columns_for_merge,right_on=columns_for_merge)

filter_all_maps = df_performance_merged['map'] != 'All maps' 

df_performance_merged = df_performance_merged[filter_all_maps]

df_performance_merged.to_csv('peformance_merged.csv')

C:\Users\lrktl\AppData\Local\Temp\ipykernel_4384\2063687743.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_full_round_to_export['powerBikey'] = df_full_round_to_export[key_columns].astype(str).agg('_'.join,axis=1)


In [1]:
def text_to_index(df,name):
    name_id = name+'_id'
    df[name_id] = df.index
    df[name_id] = name + '_' + df[name_id].astype(str)
    return df

In [4]:
#regiones
regiones = {'region':['Pacific','Americas','China','EMEA','Global']}

df_region = pd.DataFrame(regiones)

df_region = text_to_index(df_region,'region')

In [8]:
#Torneos
torneos = {'tournament':['Kickoff Americas', 'Kickoff Pacific', 'Kickoff EMEA', 'Kickoff China', 'Champions Tour 2025: Masters Bangkok'],
           'region' : ['region_1','region_0','region_3','region_2','region_4']
           }

df_tournament = pd.DataFrame(torneos)

df_tournament = text_to_index(df_tournament,'tournament')

In [9]:
teams = {'team_name': [team_name for team_name in tricode_to_fullname.values()],
         'region': ['region_2','region_3','region_0','region_1','region_3','region_0','region_1','region_2'] #OJO CON ESTO HARDCODEADO
         } 

df_teams = pd.DataFrame(teams)

df_teams = text_to_index(df_teams,'team')

In [10]:
df_players = pd.read_csv('playerstats.csv') #El player id es malo probablemente habria que cambiarlo con range y despues meter 

df_players = df_players[['player','team']].drop_duplicates(subset=['player'])

df_players = text_to_index(df_players,'player')

map_team_name = dict(zip(df_teams['team_name'], df_teams['team_id']))

df_players['team'] = df_players['team'].map(map_team_name)

In [11]:
df_picks = pd.read_csv('picks.csv')

amount_matches = len(df_picks.drop_duplicates(subset='date'))

match_tournament_dict = {'touranment_id':['tournament_4' for x in range(0,amount_matches)], #Ojo con esto medio hardcodeado
                         'draft_id' : ['draft_'+str(x) for x in range(0,amount_matches)]
                         }

df_match_tournaments = pd.DataFrame(match_tournament_dict)

In [12]:
df_picks = pd.read_csv('picks.csv')

draft_id_dict = dict(zip(df_picks['date'].unique(),match_tournament_dict['draft_id']))

df_picks['draft_id'] = df_picks['date'].map(draft_id_dict)

def create_draft_table(df):
    filas_transformadas = []

    for index, row in df.iterrows():
        picks = [
            row['team_1_select_1'], row['team_2_select_1'],
            row['team_1_select_2'], row['team_2_select_2'],
            row['team_1_select_3'], row['team_2_select_3'],
            row['decider']
        ]

    # Añadir cada selección como una nueva fila
        for pick_num, map_name in enumerate(picks, start=1):
            filas_transformadas.append({
                'team': row['team'],
                'draft_id': row['draft_id'],
                'order': row['order'],
                'bo': row['bo'],
                'pick': pick_num,
                'map_name': map_name
            })

    # Crear el nuevo DataFrame con las filas transformadas
    nuevo_df = pd.DataFrame(filas_transformadas)
    return nuevo_df

df_drafts = create_draft_table(df_picks)

In [13]:
df_bo = df_drafts[['draft_id','bo']].drop_duplicates(subset='draft_id')

dict_id_bo = dict(zip(df_bo['draft_id'],df_bo['bo']))

matches_dict = {'draft':[],'match_id':[]}
count = 0

for draft_id in df_match_tournaments['draft_id']:
    amount_matches = int(dict_id_bo[draft_id])
    for trio in range(1,amount_matches+1):
        matches_dict['draft'].append(draft_id)
        matches_dict['match_id'].append('match_'+ str(count))
        count += 1

df_matches = pd.DataFrame(matches_dict)

In [ ]:
df_stats = pd.read_csv('Statsamer1.csv')

df_stats['match_id'] = df_matches['match_id']

column_order_map_stats = ['match_id','team','rival','map','own_pistol_tt','own_pistol_ct','own_score_tt','rival_score_tt','own_score_ct','rival_score_ct','date']

df_map_stats = df_stats[column_order_map_stats]

df_map_stats = df_map_stats.rename(columns={'team':'teamA','rival':'teamB','own_pistol_tt':'teamA_pistol_tt','own_pistol_ct':'teamA_pistol_ct','own_score_tt':'teamA_score_tt','rival_score_tt':'teamB_score_tt','own_score_ct':'teamA_score_ct','rival_score_ct':'teamB_score_ct'})

def check_played(rounds_played):
    if rounds_played != 0:
        return 1
    else:
        return 0
    
df_map_stats['played'] = df_map_stats['teamA_score_tt']+df_map_stats['teamB_score_tt']+df_map_stats['teamA_score_ct'] +df_map_stats['teamB_score_ct']

df_map_stats['played'] = df_map_stats['played'].apply(check_played)

In [ ]:
df_round_stats = pd.read_csv('round_detail_merged.csv')

#Hay columnas de mas date, event y powerBikey seguro, event lo tendria que cambiar por tournament y aca tengo la duda des si es necesario poner esa de torunament o si el matchid unico es suficiente para no poner ningun torunament

df_round_stats_match_id = pd.merge(df_round_stats, df_map_stats[['match_id','map','date']], on=['date','map'])

df_round_stats_ids = pd.merge(df_round_stats_match_id,df_tournament[['tournament','tournament_id']], left_on='event', right_on=['tournament'])

columns_round_stats = ['teamA','teamB','map','side','round','team_a_economy','team_b_economy','rndA','rndB','winCon','map_order','match_id','tournament_id','date']
df_round_stats_ids = df_round_stats_ids[columns_round_stats]

In [ ]:
df_player_stats = pd.read_csv('peformance_merged.csv')

df_player_stats_ids = pd.merge(df_player_stats,df_players,left_on='player',right_on='player')

df_player_stats_ids


In [ ]:
maps_list = list(df_map_stats['map'].unique())

df_maps_id = pd.DataFrame(maps_list)
df_maps_id = df_maps_id.rename(columns={0:'map_name'})

df_maps_id = text_to_index(df_maps_id,'map')



In [ ]:
df_player_stats.to_csv(path_or_buf='table_player_stats.csv',index=False)
df_round_stats_ids.to_csv(path_or_buf='table_round_stats.csv',index=False)
df_map_stats.to_csv(path_or_buf='table_map_stats.csv',index=False)
df_matches.to_csv(path_or_buf='table_matches.csv',index=False)
df_drafts.to_csv(path_or_buf='table_drafts.csv',index=False)
df_match_tournaments.to_csv(path_or_buf='table_match_tournament.csv',index=False)
df_players.to_csv(path_or_buf='table_players.csv',index=False)
df_teams.to_csv(path_or_buf='table_teams.csv',index=False)
df_tournament.to_csv(path_or_buf='table_tournaments.csv',index=False)
df_region.to_csv(path_or_buf='table_regions.csv',index=False)
df_maps_id.to_csv(path_or_buf='table_maps.csv',index=False)


In [82]:
df_maps_id.to_csv(path_or_buf='table_maps.csv',index=False)
df_maps_id

,map_name,map_id
0,Split,map_0
1,Pearl,map_1
2,Bind,map_2
3,Lotus,map_3
4,Haven,map_4
5,Abyss,map_5
6,Fracture,map_6
